### 1. 라이브 커머스 다시 보기 영상 다운 후 mp3 파일로 변경

In [4]:
from moviepy.editor import *

video = VideoFileClip('./test_video.mkv')

video.audio.write_audiofile('./test_video.mp3')

MoviePy - Writing audio in ./test_video.mp3


MoviePy - Done.


### 2. MP3 파일을 텍스트로 변환

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda" if torch.cuda.is_available() else 'cpu'
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"
# model_id = "openai/whisper-medium"


model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)

model.to(device)
processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens = 64,
    chunk_length_s = 10,
    batch_size = 8,
    return_timestamps = True,
    torch_dtype=torch_dtype,
    device=device
)

speech_output_path = './test_video.mp3'
result_openai = pipe(speech_output_path)

with open('stt_file.txt', 'w', encoding='utf-8') as stt_file:
    stt_file.write(result_openai["text"])

c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\transformers\models\whisper\generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


### 3. 텍스트 속 제품 내용 추출

In [11]:
import os
from langchain.document_loaders import TextLoader
from langchain.chains.summarize import load_summarize_chain
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()

# 문서 로딩
loader = TextLoader('./stt_file.txt', encoding='utf-8')
document = loader.load()

# 텍스트 분할 설정
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=3000, chunk_overlap=300)

def extract_product_info(review):
    docs = text_splitter.create_documents([review])

    # 제품 정보를 추출하기 위한 프롬프트 템플릿
    map_prompt_template = '''다음 텍스트에서 제품 정보를 추출해주세요:
    - 제품명
    - 주요 성분
    - 제품 효과
    - 사용 방법
    - 프로모션 정보
    - 배송일 정보
    - 가격 정보
    - 고객 후기
    - 유통기한/보관 정보
    - 구성품 정보
    - 대상 고객
    - 특별 주의사항
    - 배송비 및 조건
    - 제품의 세부 사용법
    - 성분의 상세 설명과 효과
    - 스트리머의 사용 후기
    텍스트: {text}
    '''
    combine_prompt_template = '''아래의 텍스트에서 추출된 제품 정보를 종합해 주세요:
    {text}
    '''

    MAP_PROMPT = PromptTemplate(template=map_prompt_template, input_variables=["text"])
    COMBINE_PROMPT = PromptTemplate(template=combine_prompt_template, input_variables=["text"])

    chain = load_summarize_chain(ChatOpenAI(temperature=0, model_name='gpt-4o-mini'),
                                 chain_type="map_reduce", return_intermediate_steps=True,
                                 map_prompt=MAP_PROMPT, combine_prompt=COMBINE_PROMPT)

    return chain({"input_documents": docs}, return_only_outputs=True)

# 추출 함수 실행
extracted_info = extract_product_info(document[0].page_content)
print(extracted_info)

# 제품명 추출
output_text = extracted_info['output_text']
lines = output_text.split("\n")

# "제품명"이라는 단어를 포함한 줄을 찾아서 제품명을 추출
product_name = None
for line in lines:
    if "제품명" in line:
        product_name = line.split(": ")[1]
        break

if product_name:
    print(f"추출된 제품명: {product_name}")
else:
    print("제품명을 추출하지 못했습니다.")


{'intermediate_steps': ['제품 정보 추출 결과:\n\n- **제품명**: 아미퓨어 갈락토 세럼 미스트\n- **주요 성분**: 갈락토미세스 발효 여과물 (90% 함유)\n- **제품 효과**: 보습, 피부 결 개선, 속보습, 광채 효과, 각질 케어\n- **사용 방법**: 얼굴에 가볍게 뿌리거나 화장솜에 적셔서 팩처럼 사용\n- **프로모션 정보**: 1,000원 할인 쿠폰 제공, 2개 구매 시 36% 할인\n- **배송일 정보**: 무료배송\n- **가격 정보**: 24,500원 (할인 후 23,500원), 2개 세트 45,000원 (할인 후 44,000원)\n- **고객 후기**: 수분감 유지, 피부결 개선, 가벼운 사용감, 각질 케어 효과\n- **유통기한/보관 정보**: 정보 없음\n- **구성품 정보**: 세니타이저 2개 (1개 구매 시), 4개 (2개 구매 시)\n- **대상 고객**: 남녀노소, 피부 고민이 있는 모든 고객\n- **특별 주의사항**: 예민한 피부도 사용 가능, 손대지 않고 뿌려서 사용 권장\n- **배송비 및 조건**: 무료배송\n- **제품의 세부 사용법**: 스킨팩으로 사용 가능, 메이크업 전후에 사용 가능\n- **성분의 상세 설명과 효과**: 갈락토미세스 성분이 피부 결을 부드럽게 하고, 속보습을 제공하며, 광채를 더함\n- **스트리머의 사용 후기**: 피부가 예민할 때도 사용 가능, 즉각적인 수분 공급 효과\n\n이 정보는 제품에 대한 전반적인 이해를 돕기 위해 요약된 것입니다.'], 'output_text': '**제품 정보 요약: 아미퓨어 갈락토 세럼 미스트**\n\n- **제품명**: 아미퓨어 갈락토 세럼 미스트\n- **주요 성분**: 갈락토미세스 발효 여과물 (90% 함유)\n- **효과**: \n  - 보습\n  - 피부 결 개선\n  - 속보습\n  - 광채 효과\n  - 각질 케어\n- **사용 방법**: \n  - 얼굴에 가볍게 뿌리거나\n  - 화장솜에 적셔서 팩처럼 사용\n- **

### 4. 제품 추천

In [10]:
import requests

# 네이버 쇼핑 API를 사용한 제품 검색 함수
client_id = 'juA2ijfDhrLAObXfhY6_'  # 네이버 개발자 센터에서 발급받은 Client ID
client_secret = 'BQDR8znGZI'  # 네이버 개발자 센터에서 발급받은 Client Secret

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

def search_product(query):
    url = f"https://openapi.naver.com/v1/search/shop.json?query={query}&display=10&start=1&sort=sim"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# 추출된 제품명을 사용하여 네이버 쇼핑 API로 검색
result = search_product(product_name)

if result:
    for item in result['items']:
        print(f"Title: {item['title']}")
        print(f"Link: {item['link']}")
        print(f"Lprice: {item['lprice']}원")
        print("-----")

Title: <b>아미퓨어 갈락토 세럼미스트</b> 100ml  1개
Link: https://link.coupang.com/re/PCSNAVERPCSDP?pageKey=8051622723&ctag=8051622723&lptag=I22583823214&itemId=22583823214&vendorItemId=89625581058&spec=10305199
Lprice: 35000원
-----
Title: <b>아미퓨어 갈락토 세럼미스트</b> 100ml - 갈락토미세스 90% 함유 영양 수분 보습 수분충전 안개분사 간편휴대
Link: https://smartstore.naver.com/main/products/10227810002
Lprice: 35000원
-----


### ===============================================================================

### 5. 코드 함수화

In [ ]:
import os
import time
import requests
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from langchain.document_loaders import TextLoader
from langchain.chains.summarize import load_summarize_chain
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

def init_webdriver(headless=False):
    options = Options()
    options.headless = headless
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def fetch_shoppinglive_data(url, driver=None):
    if driver is None:
        driver = init_webdriver()
    driver.get(url)
    time.sleep(3)  # 페이지 로드 대기
    # 데이터 수집 로직 추가 필요
    driver.quit()

def transcribe_audio(speech_output_path):
    device = "cuda" if torch.cuda.is_available() else 'cpu'
    model_id = "openai/whisper-large-v3"
    
    model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch.float16 if device == 'cuda' else torch.float32)
    processor = AutoProcessor.from_pretrained(model_id)
    
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model.to(device),
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=64,
        chunk_length_s=10,
        batch_size=8,
        return_timestamps=True,
        torch_dtype=model.dtype,
        device=device
    )
    
    return pipe(speech_output_path)["text"]

def load_text_document(file_path):
    return TextLoader(file_path, encoding='utf-8').load()

def extract_product_info(text):
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=3000, chunk_overlap=300)

    map_prompt_template = '''다음 텍스트에서 제품 정보를 추출해주세요:
    - 제품명
    - 주요 성분
    - 제품 효과
    - 사용 방법
    - 프로모션 정보
    - 배송일 정보
    - 가격 정보
    - 고객 후기
    - 유통기한/보관 정보
    - 구성품 정보
    - 대상 고객
    - 특별 주의사항
    - 배송비 및 조건
    - 제품의 세부 사용법
    - 성분의 상세 설명과 효과
    - 스트리머의 사용 후기
    텍스트: {text}
    '''
    
    combine_prompt_template = '''아래의 텍스트에서 추출된 제품 정보를 종합해 주세요:
    {text}
    '''

    MAP_PROMPT = PromptTemplate(template=map_prompt_template, input_variables=["text"])
    COMBINE_PROMPT = PromptTemplate(template=combine_prompt_template, input_variables=["text"])

    chain = load_summarize_chain(ChatOpenAI(temperature=0, model_name='gpt-4o-mini'),
                                 chain_type="map_reduce", return_intermediate_steps=True,
                                 map_prompt=MAP_PROMPT, combine_prompt=COMBINE_PROMPT)

    docs = text_splitter.create_documents([text])
    return chain({"input_documents": docs}, return_only_outputs=True)

def extract_product_name(text):
    for line in text.split("\n"):
        if "제품명" in line:
            return line.split(": ")[1]
    return None

def search_product(query, client_id, client_secret):
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    url = f"https://openapi.naver.com/v1/search/shop.json?query={query}&display=10&start=1&sort=sim"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

def main(speech_output_path, client_id, client_secret):
    text = transcribe_audio(speech_output_path)
    document = load_text_document('./stt_file.txt')
    
    extracted_info = extract_product_info(document[0].page_content)
    product_name = extract_product_name(extracted_info['output_text'])
    
    if not product_name:
        print("제품명을 추출하지 못했습니다.")
        return

    print(f"추출된 제품명: {product_name}")
    
    result = search_product(product_name, client_id, client_secret)
    if result:
        for item in result['items']:
            print(f"Title: {item['title']}")
            print(f"Link: {item['link']}")
            print(f"Lprice: {item['lprice']}원")
            print("-----")

if __name__ == "__main__":
    client_id = '개인 API ID로 수정'
    client_secret = 'API 비밀번호 수정'
    speech_output_path = './test_video.mp3'
    main(speech_output_path, client_id, client_secret)
